# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# uncomment to show current Jupyter notebook working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Queries

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from os import environ
from cassandra.cluster import Cluster

# set host variable based on your local environment
if environ.get("UDACITY_DATA_ENGINEER_CASSANDRA_CLUSTER"):
    host = environ.get("UDACITY_DATA_ENGINEER_CASSANDRA_CLUSTER")
    
try: 
    cluster = Cluster([host]) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

/opt/conda/envs/udacity-data-engineering-nano/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS song_user_history
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('song_user_history')
except Exception as e:
    print(e)

##  Queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
## Query 1:  Return the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS song_session_listen_history"
query = query + "(sessionId INT, itemInSession SMALLINT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY ((sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Insert data into song_session_listen_history

file = 'event_datafile_new.csv'

# Loop through CSV file and insert data into Cassandra
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statement 
        query = "INSERT INTO song_session_listen_history (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Try INSERT statement
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

In [10]:
## Query 1:  Return the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, length FROM Song_session_listen_history WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2

Return the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
## Query 2: Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userId = 10, sessionId = 182

query = "CREATE TABLE IF NOT EXISTS artist_song_user_session_listen_history"
# note: itemInSession added to primary key to order ascending by this value based on query requirements
query = query + "(userId INT , sessionId INT, itemInSession SMALLINT, artist TEXT, song TEXT, user TEXT, PRIMARY KEY ((userid, sessionid), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [12]:
# Insert data into artist_song_user_session_listen_history

file = 'event_datafile_new.csv'

# Loop through CSV file and insert data into Cassandra
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statement 
        query = "INSERT INTO artist_song_user_session_listen_history (userId, sessionId, itemInSession, artist, song, user)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Try INSERT statement
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))
        except Exception as e:
            print(e)

In [13]:
## Query 2: Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userId = 10, sessionId = 182
query = "SELECT artist, song, user FROM artist_song_user_session_listen_history WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3

Return the user name (first and last) in the music app history for those who listened to the song 'All Hands Against His Own'

In [14]:
## Query 3: Return the user name (first and last) in the music app history for those who listened to the song 'All Hands Against His Own'

# song and userId may be repeated for multiple song listens by same user
# not an issue since query is to just return unique users that have listened to the song
# did not use user's name in primary key since multiple users could have same name and have listened to same song

query = "CREATE TABLE IF NOT EXISTS user_song_listen_history"
query = query + "(song TEXT, userId INT, user TEXT, PRIMARY KEY ((song), userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
# Insert data into user_song_listen_history

file = 'event_datafile_new.csv'

# Loop through CSV file and insert data into Cassandra
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statement 
        query = "INSERT INTO user_song_listen_history (song, userId, user)"
        query = query + " VALUES (%s, %s, %s)"
        ## Try INSERT statement
        try:
            session.execute(query, (line[9], int(line[10]), line[1] + " " + line[4]))
        except Exception as e:
            print(e)

In [16]:
## Query 3: Return the user name (first and last) in the music app history for those who listened to the song 'All Hands Against His Own'

query = "SELECT user FROM user_song_listen_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
# drop table song_session_listen_history
query = "DROP TABLE IF EXISTS song_session_listen_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# drop table artist_song_user_session_listen_history
query = "DROP TABLE IF EXISTS artist_song_user_session_listen_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# drop table user_song_listen_history
query = "DROP TABLE IF EXISTS user_song_listen_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()